In [1]:
import sys
import os

# Move up one level so Python can find config.py
sys.path.append(os.path.abspath(".."))


import psycopg2
from config import get_db_connection
import pandas as pd


In [2]:
from config import get_db_connection

# Test the connection
conn = get_db_connection()
if conn:
    print("✅ Database connection successful!")
    conn.close()
else:
    print("❌ Database connection failed. Check your credentials in config.py.")


✅ Database connection successful!


In [4]:
conn = get_db_connection()
df = pd.read_sql("SELECT * FROM stg_superstore", conn)
conn.close()

df.head()


C:\Users\Pranesh\AppData\Local\Temp\ipykernel_24472\3424497431.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM stg_superstore", conn)


,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales
0,1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


Insert into Hubs

In [5]:
# Hub: Customer
customers = df[['customer_id']].drop_duplicates()
# Hub: Product
products = df[['product_id']].drop_duplicates()
# Hub: Order
orders = df[['order_id']].drop_duplicates()

In [6]:
conn = get_db_connection()
cursor = conn.cursor()

In [7]:

try: 
    # Insert into hub_customer
    for row in customers.itertuples(index=False):
        cursor.execute("INSERT INTO hub_customer (customer_id) VALUES (%s) ON CONFLICT DO NOTHING", (row.customer_id,))

except psycopg2.Error as e:
    print(e)

In [8]:

try: 
    # Insert into hub_product
    for row in products.itertuples(index=False):
        cursor.execute("INSERT INTO hub_product (product_id) VALUES (%s) ON CONFLICT DO NOTHING", (row.product_id,))
    print("✅ Inserted data into the 'hub_product' table successfully!")
except psycopg2.Error as e:
    print(e)

✅ Inserted data into the 'hub_product' table successfully!


In [9]:

try: 
    # Insert into hub_order
    for row in orders.itertuples(index=False):
        cursor.execute("INSERT INTO hub_order (order_id) VALUES (%s) ON CONFLICT DO NOTHING", (row.order_id,))
    print("✅ Inserted data into the 'hub_order' table successfully!")
except psycopg2.Error as e:
    print(e)

✅ Inserted data into the 'hub_order' table successfully!


In [10]:
conn.commit()
print("✅ Hubs populated.")

✅ Hubs populated.


Build & Insert into Link Table

In [11]:
link_df = df[['order_id', 'customer_id', 'product_id']].drop_duplicates()

In [12]:

try: 
    # Insert into link_order_customer_product
    for row in link_df.itertuples(index=False):
        cursor.execute("""
            INSERT INTO link_order_customer_product (order_id, customer_id, product_id)
            VALUES (%s, %s, %s)
            ON CONFLICT DO NOTHING
        """, (row.order_id, row.customer_id, row.product_id))
    print("✅ Inserted data into the 'link_order_customer_product' table successfully!")
except psycopg2.Error as e:
    print(e)

✅ Inserted data into the 'link_order_customer_product' table successfully!


In [13]:
conn.commit()
print("✅ Link table populated.")

✅ Link table populated.


Insert into Satellites

In [14]:
sat_cust = df[['customer_id', 'customer_name', 'segment', 'country', 'city', 'state', 'postal_code', 'region']].drop_duplicates()


In [15]:

try: 
    # Insert into sat_customer
    for row in sat_cust.itertuples(index=False):
        cursor.execute("""
            INSERT INTO sat_customer (
                customer_id, customer_name, segment, country, city, state, postal_code, region
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT DO NOTHING
        """, tuple(row))
    print("✅ Inserted data into the 'sat_customer' table successfully!")
except psycopg2.Error as e:
    print(e)

✅ Inserted data into the 'sat_customer' table successfully!


In [16]:
conn.commit()

In [17]:
sat_prod = df[['product_id', 'category', 'sub_category', 'product_name']].drop_duplicates()

In [18]:

try: 
    # Insert into sat_product
    for row in sat_prod.itertuples(index=False):
        cursor.execute("""
            INSERT INTO sat_product (
                product_id, category, sub_category, product_name
            ) VALUES (%s, %s, %s, %s)
            ON CONFLICT DO NOTHING
        """, tuple(row))
    print("✅ Inserted data into the 'sat_product' table successfully!")
except psycopg2.Error as e:
    print(e)

✅ Inserted data into the 'sat_product' table successfully!


In [19]:
conn.commit()

In [20]:
sat_order = df[['order_id', 'order_date', 'ship_date', 'ship_mode']].drop_duplicates()

In [21]:

try: 
    # Insert into sat_order
    for row in sat_order.itertuples(index=False):
        cursor.execute("""
            INSERT INTO sat_order (
                order_id, order_date, ship_date, ship_mode
            ) VALUES (%s, %s, %s, %s)
            ON CONFLICT DO NOTHING
        """, tuple(row))
    print("✅ Inserted data into the 'sat_order' table successfully!")
except psycopg2.Error as e:
    print(e)

✅ Inserted data into the 'sat_order' table successfully!


In [22]:
conn.commit()

In [23]:
# Pull link table to join
link_df = pd.read_sql("""
    SELECT link_id, order_id, customer_id, product_id
    FROM link_order_customer_product
""", get_db_connection())

C:\Users\Pranesh\AppData\Local\Temp\ipykernel_24472\3528983373.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  link_df = pd.read_sql("""


In [24]:
# Join to get link_id per row
merged = pd.merge(df, link_df, on=['order_id', 'customer_id', 'product_id'])

In [ ]:
# Insert into satellite
conn = get_db_connection()
cursor = conn.cursor()

In [25]:

try: 
    # Insert into sat_orderitem
    for row in merged[['link_id', 'sales']].itertuples(index=False):
        cursor.execute("""
            INSERT INTO sat_orderitem (link_id, sales)
            VALUES (%s, %s)
        """, tuple(row))
    print("✅ Inserted data into the 'sat_orderitem' table successfully!")
except psycopg2.Error as e:
    print(e)

✅ Inserted data into the 'sat_orderitem' table successfully!


In [26]:
conn.commit()
cursor.close()
conn.close()
print("✅ All Satellites populated.")

✅ All Satellites populated.
